In [25]:
library(ggplot2)
library(precrec)
library(readr)
library(reticulate)
sklearn <- import("sklearn.metrics")

In [26]:
calculate <- function(data, result_path, column) {
  epitope_groups <- split(data, data[[column]])
  roc_prc_values <- data.frame(
    epitope = character(),
    roc_auc = numeric(),
    prc_auc = numeric(),
    accuracy = numeric(),
    precision = numeric(),
    recall = numeric(),
    specificity = numeric(),
    mcc = numeric(),
    f1 = numeric(),
    stringsAsFactors = FALSE
  )
  
  for (epitope in names(epitope_groups)) {
    subset_data <- epitope_groups[[epitope]]
    accuracy <- sklearn$accuracy_score(subset_data$y_true, subset_data$y_pred)
    precision <- sklearn$precision_score(subset_data$y_true, subset_data$y_pred)
    recall <- sklearn$recall_score(subset_data$y_true, subset_data$y_pred)
    specificity <- sklearn$recall_score(subset_data$y_true, subset_data$y_pred, pos_label=0)
    mcc <- sklearn$matthews_corrcoef(subset_data$y_true, subset_data$y_pred)
    f1 <- sklearn$f1_score(subset_data$y_true, subset_data$y_pred)

    sscurves <- evalmod(scores = subset_data$y_prob, labels = subset_data$y_true)
    auc_values <- auc(sscurves)
    roc_auc <- subset(auc_values, curvetypes == "ROC")$aucs
    prc_auc <- subset(auc_values, curvetypes == "PRC")$aucs
      
    roc_prc_values <- rbind(
      roc_prc_values,
      data.frame(
        epitope = epitope,
        roc_auc = roc_auc,
        prc_auc = prc_auc,
        accuracy = accuracy,
        precision = precision,
        recall = recall,
        specificity = specificity,
        mcc = mcc,
        f1 = f1,
        stringsAsFactors = FALSE
      )
    )
  }
  
  accuracy_all <- sklearn$accuracy_score(data$y_true, data$y_pred)
  precision_all <- sklearn$precision_score(data$y_true, data$y_pred)
  recall_all <- sklearn$recall_score(data$y_true, data$y_pred)
  specificity_all <- sklearn$recall_score(data$y_true, data$y_pred, pos_label=0)
  mcc_all <- sklearn$matthews_corrcoef(data$y_true, data$y_pred)
  f1_all <- sklearn$f1_score(data$y_true, data$y_pred)
  
  sscurves_all <- evalmod(scores = data$y_prob, labels = data$y_true)
  auc_values_all <- auc(sscurves_all)
  roc_auc_all <- subset(auc_values_all, curvetypes == "ROC")$aucs
  prc_auc_all <- subset(auc_values_all, curvetypes == "PRC")$aucs
  
  roc_prc_values <- rbind(
    roc_prc_values,
    data.frame(
      epitope = "all_values",
      roc_auc = roc_auc_all,
      prc_auc = prc_auc_all,
      accuracy = accuracy_all,
      precision = precision_all,
      recall = recall_all,
      specificity = specificity_all,
      mcc = mcc_all,
      f1 = f1_all,
      stringsAsFactors = FALSE
    )
  )
  
  output_file_path <- paste0(result_path, "all_result.csv")
  write.csv(roc_prc_values, file = output_file_path, row.names = FALSE)
  print(paste("Saved File：", output_file_path))
}


process_files <- function(database, name1, name, base_path,column) {
  for (i in database) {
    for (j in name) {
      for (k in name1) {
        result_path <- paste0(base_path, i, "/", k, "_", j)
        file_path <- paste0(result_path, "probability.csv")
        if (file.exists(file_path)) {
          data <- read_csv(file_path, show_col_types = FALSE)
          calculate(data, result_path,column)
        } else {
          print(paste("File does not exist：", file_path))
        }
      }
    }
  }
}
pair300_process_files <- function(database, name1, name,pair300, base_path,column) {
  for (i in database) {
    for (j in name) {
      for (k in name1) {
          for (l in pair300) {
            result_path <- paste0(base_path, l, "/",i, "/", k, "_", j)
            file_path <- paste0(result_path, "probability.csv")
            if (file.exists(file_path)) {
              data <- read_csv(file_path, show_col_types = FALSE)
              calculate(data, result_path,column)
            } else {
              print(paste("File does not exist：", file_path))
            }
          }
        }
      }
    }
}

In [27]:
calculate_ERGO_ae <- function(data, result_path, column) {
  epitope_groups <- split(data, data[[column]])
  roc_prc_values <- data.frame(
    epitope = character(),
    roc_auc = numeric(),
    prc_auc = numeric(),
    accuracy = numeric(),
    precision = numeric(),
    recall = numeric(),
    specificity = numeric(),
    mcc = numeric(),
    f1 = numeric(),
    stringsAsFactors = FALSE
  )
  
  for (epitope in names(epitope_groups)) {
    subset_data <- epitope_groups[[epitope]]
    if (length(unique(subset_data$y_true)) == 1) {
      accuracy <- 0.5
      precision <- 0.5
      recall <- 0.5
      specificity <- 0.5
      mcc <- 0.5
      f1 <- 0.5
      roc_auc <- 0.5
      prc_auc <- 0.5
    } else {
      accuracy <- sklearn$accuracy_score(subset_data$y_true, subset_data$y_pred)
      precision <- sklearn$precision_score(subset_data$y_true, subset_data$y_pred)
      recall <- sklearn$recall_score(subset_data$y_true, subset_data$y_pred)
      specificity <- sklearn$recall_score(subset_data$y_true, subset_data$y_pred, pos_label=0)
      mcc <- sklearn$matthews_corrcoef(subset_data$y_true, subset_data$y_pred)
      f1 <- sklearn$f1_score(subset_data$y_true, subset_data$y_pred)
      
      sscurves <- evalmod(scores = subset_data$y_prob, labels = subset_data$y_true)
      auc_values <- auc(sscurves)
      roc_auc <- subset(auc_values, curvetypes == "ROC")$aucs
      prc_auc <- subset(auc_values, curvetypes == "PRC")$aucs
    }

    roc_prc_values <- rbind(
      roc_prc_values,
      data.frame(
        epitope = epitope,
        roc_auc = roc_auc,
        prc_auc = prc_auc,
        accuracy = accuracy,
        precision = precision,
        recall = recall,
        specificity = specificity,
        mcc = mcc,
        f1 = f1,
        stringsAsFactors = FALSE
      )
    )
  }

  if (length(unique(data$y_true)) == 1) {
    accuracy_all <- 0.5
    precision_all <- 0.5
    recall_all <- 0.5
    specificity_all <- 0.5
    mcc_all <- 0.5
    f1_all <- 0.5
    roc_auc_all <- 0.5
    prc_auc_all <- 0.5
  } else {
    accuracy_all <- sklearn$accuracy_score(data$y_true, data$y_pred)
    precision_all <- sklearn$precision_score(data$y_true, data$y_pred)
    recall_all <- sklearn$recall_score(data$y_true, data$y_pred)
    specificity_all <- sklearn$recall_score(data$y_true, data$y_pred, pos_label=0)
    mcc_all <- sklearn$matthews_corrcoef(data$y_true, data$y_pred)
    f1_all <- sklearn$f1_score(data$y_true, data$y_pred)
    
    sscurves_all <- evalmod(scores = data$y_prob, labels = data$y_true)
    auc_values_all <- auc(sscurves_all)
    roc_auc_all <- subset(auc_values_all, curvetypes == "ROC")$aucs
    prc_auc_all <- subset(auc_values_all, curvetypes == "PRC")$aucs
  }
  
  roc_prc_values <- rbind(
    roc_prc_values,
    data.frame(
      epitope = "all_values",
      roc_auc = roc_auc_all,
      prc_auc = prc_auc_all,
      accuracy = accuracy_all,
      precision = precision_all,
      recall = recall_all,
      specificity = specificity_all,
      mcc = mcc_all,
      f1 = f1_all,
      stringsAsFactors = FALSE
    )
  )
  
  output_file_path <- paste0(result_path, "all_result.csv")
  write.csv(roc_prc_values, file = output_file_path, row.names = FALSE)
  print(paste("Saved File：", output_file_path))
}

process_files_ERGO_ae <- function(database, name1, name, base_path,column) {
  for (i in database) {
    for (j in name) {
      for (k in name1) {
        result_path <- paste0(base_path, i, "/", k, "_", j)
        file_path <- paste0(result_path, "probability.csv")
        if (file.exists(file_path)) {
          data <- read_csv(file_path, show_col_types = FALSE)
          calculate_ERGO_ae(data, result_path,column)
        } else {
          print(paste("File does not exist：", file_path))
        }
      }
    }
  }
}

In [28]:
process_original <- function(names, mes, nums = NULL, base_path, column) {
  for (name in names) {
    for (me in mes) {
      if (!is.null(nums)) {
        for (num in nums) {
          path <- paste0(base_path, num, "/", name, "_", me)
          data_path <- paste0(path, "probability.csv")
          if (file.exists(data_path)) {
            data <- read.csv(data_path, stringsAsFactors = FALSE)
            calculate(data, path, column)
          } else {
            warning("The file does not exist: ", data_path)
          }
        }
      } else {
        path <- paste0(base_path, name, "_", me)
        data_path <- paste0(path, "probability.csv")
        if (file.exists(data_path)) {
          data <- read.csv(data_path, stringsAsFactors = FALSE)
          calculate(data, path, column)
        } else {
          warning("The file does not exist: ", data_path)
        }
      }
    }
  }
}

In [29]:
calculate_top5_auc <- function(data, result_path, column) {
  epitope_groups <- split(data, data[[column]])
  roc_prc_values <- data.frame(
    epitope = character(),
    roc_auc = numeric(),
    prc_auc = numeric(),
    accuracy = numeric(),
    precision = numeric(),
    recall = numeric(),
    specificity = numeric(),
    mcc = numeric(),
    f1 = numeric(),
    stringsAsFactors = FALSE
  )
  
  for (epitope in names(epitope_groups)) {
    subset_data <- epitope_groups[[epitope]]
    accuracy <- sklearn$accuracy_score(subset_data$y_true, subset_data$y_pred)
    precision <- sklearn$precision_score(subset_data$y_true, subset_data$y_pred)
    recall <- sklearn$recall_score(subset_data$y_true, subset_data$y_pred)
    specificity <- sklearn$recall_score(subset_data$y_true, subset_data$y_pred, pos_label = 0)
    mcc <- sklearn$matthews_corrcoef(subset_data$y_true, subset_data$y_pred)
    f1 <- sklearn$f1_score(subset_data$y_true, subset_data$y_pred)

    sscurves <- evalmod(scores = subset_data$y_prob, labels = subset_data$y_true)
    auc_values <- auc(sscurves)
    roc_auc <- subset(auc_values, curvetypes == "ROC")$aucs
    prc_auc <- subset(auc_values, curvetypes == "PRC")$aucs
      
    roc_prc_values <- rbind(
      roc_prc_values,
      data.frame(
        epitope = epitope,
        roc_auc = roc_auc,
        prc_auc = prc_auc,
        accuracy = accuracy,
        precision = precision,
        recall = recall,
        specificity = specificity,
        mcc = mcc,
        f1 = f1,
        stringsAsFactors = FALSE
      )
    )
  }
  
  output_file_path <- paste0(result_path, "all_result.csv")
  write.csv(roc_prc_values, file = output_file_path, row.names = FALSE)
  print(paste("Saved File:", output_file_path))
}

top5 <- function(reap, name, number, n_estimators, base_path,column) {
  for (i in reap) {
    for (j in name) {
      for (k in number) {
        for (estimators in n_estimators) {
          path <- paste0(base_path, i, '_', j, '_', k, "_", estimators)
          data_path <- paste0(path, "probability.csv")
          if (file.exists(data_path)) {
            data <- read.csv(data_path, stringsAsFactors = FALSE)
            calculate_top5_auc(data, path, column) 
          } else {
            warning("The file does not exist: ", data_path)
          }
        }
      }
    }
  }
}

top5_no_parameter <- function(reap, name, number, base_path,column) {
  for (i in reap) {
    for (j in name) {
      for (k in number) {
          path <- paste0(base_path, i, '_', j, '_', k)
          data_path <- paste0(path, "probability.csv")
          if (file.exists(data_path)) {
            data <- read.csv(data_path, stringsAsFactors = FALSE)
            calculate_top5_auc(data, path, column) 
          } else {
            warning("The file does not exist: ", data_path)
          }
        }
      }
    }
  }

top5_TCRGP <- function(reap, name, number, base_path,column) {
  for (i in reap) {
    for (j in name) {
      for (k in number) {
          path <- paste0(base_path, i, '_', j, '_', k,'20')
          data_path <- paste0(path, "probability.csv")
          if (file.exists(data_path)) {
            data <- read.csv(data_path, stringsAsFactors = FALSE)
            calculate_top5_auc(data, path, column) 
          } else {
            warning("The file does not exist: ", data_path)
          }
        }
      }
    }
  }

top5ERGO <- function(reap, name, number, n_estimators, base_path,column,model) {
  for (i in reap) {
    for (j in name) {
      for (k in number) {
        for (estimators in n_estimators) {
          path <- paste0(base_path, i, '_', j, '_', k, "_", estimators,model)
          data_path <- paste0(path, "probability.csv")
          if (file.exists(data_path)) {
            data <- read.csv(data_path, stringsAsFactors = FALSE)
            calculate_top5_auc(data, path, column) 
          } else {
            warning("The file does not exist: ", data_path)
          }
        }
      }
    }
  }
}

# epiTCR

In [ ]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three', 'seven'),'./result/Original/no_CF/seen/', 'epitope')
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/no_CF/unseen/','epitope')

In [ ]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'epitope')


In [ ]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
n_estimators <- c(100, 200, 300, 400, 500)
base_path <- "./result/top5/seen/healthy/"
top5(reap, name, number, n_estimators, base_path,'epitope')


# TEINet

In [89]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/")
process_original(c('large_1', 'large_2', 'large_3'),c('health', 'Diseases', 'antigen_specificity'),c('three', 'seven'),'./result/Original/seen/', 'epitope')
process_original(c('large_1', 'large_2', 'large_3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','epitope')

setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/")
process_original(c('teinet_1', 'teinet_2', 'teinet_3'),c('health', 'Diseases', 'antigen_specificity'),c('three', 'seven'),'./result/Original/seen/', 'epitope')
process_original(c('teinet_1', 'teinet_2', 'teinet_3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','epitope')

[1] "Saved File： ./result/Original/seen/three/large_1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/large_1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/large_1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/large_1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/large_1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/large_1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/large_2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/large_2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/large_2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/large_2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/large_2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/large_2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Or

In [7]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"


New names:
• `` -> `...1`


...

[1] "Saved File： ./result/pair300/seen/200/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/100/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/10/Antigen_specificity/5_1_1all_result.csv"


In [57]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
epochs <- c(20, 30, 40, 50, 60)
base_path <- "./result/top5/seen/healthy/"
top5(reap, name, number, epochs, base_path,'Epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_20all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_40all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_20all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_40all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_20all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_40al

# NetTCR

In [77]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/NetTCR/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three'),'./result/Original/seen/', 'Epitope')
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','Epitope')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/unseen/1_healthall_result.csv"
[1] "Saved File： ./result/Original/unseen/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/unseen/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/unseen/2_healthall_result.csv"
[1] "Saved File： ./result/Original/unseen/2_Di

In [10]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/NetTCR/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'Epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"


New names:
• `` -> `...1`


...

[1] "Saved File： ./result/pair300/seen/200/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/100/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/10/Antigen_specificity/5_1_1all_result.csv"


In [58]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/NetTCR/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
epochs <- c(60,80,100,120,140)
base_path <- "./result/top5/seen/healthy/"
top5(reap, name, number, epochs, base_path,'Epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_80all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_120all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_140all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_80all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_120all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_140all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_80all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_260

# AttnTAP

In [88]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/AttnTAP/")
process_original(c('1', '2', '3'),c('health_vdjdb_', 'Diseases_vdjdb_', 'antigen_specificity_vdjdb_'),c('three', 'seven'),'./result/Original/seen/', 'antigen')
process_original(c('1', '2', '3'),c('health_vdjdb_', 'Diseases_vdjdb_', 'antigen_specificity_vdjdb_'), NULL, './result/Original/unseen/','antigen')

#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/AttnTAP/")
process_original(c('1', '2', '3'),c('health_mcpas_', 'Diseases_mcpas_', 'antigen_specificity_mcpas_'),c('three', 'seven'),'./result/Original/seen/', 'antigen')
process_original(c('1', '2', '3'),c('health_mcpas_', 'Diseases_mcpas_', 'antigen_specificity_mcpas_'), NULL, './result/Original/unseen/','antigen')


[1] "Saved File： ./result/Original/seen/three/1_health_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_health_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseases_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseases_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificity_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificity_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_health_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_health_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseases_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseases_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificity_vdjdb_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificity_vdjdb_all_result.csv"
[1] "Saved File：

In [11]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/AttnTAP/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'antigen')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'antigen')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'antigen')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'antigen')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'antigen')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'antigen')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'antigen')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'antigen')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'antigen')


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"


New names:
• `` -> `...1`


...

[1] "Saved File： ./result/pair300/seen/200/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/100/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/10/Antigen_specificity/5_1_1all_result.csv"


In [59]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/AttnTAP/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
epochs <- c(10,15,25,35,45)
base_path <- "./result/top5/seen/healthy/"
top5(reap, name, number, epochs, base_path,'antigen')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_10all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_15all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_25all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_35all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_45all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_10all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_15all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_25all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_35all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_45all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_10all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_15all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_25al

# TPBTE

In [86]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TPBTE/")
process_original(c('VDJdb_1', 'VDJdb_2', 'VDJdb_3'),c('health', 'Diseases', 'antigen_specificity'),c('three'),'./result/Original/seen/', 'Epitope')
process_original(c('VDJdb_1', 'VDJdb_2', 'VDJdb_3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','Epitope')

process_original(c('McPAS_1', 'McPAS_2', 'McPAS_3'),c('health', 'Diseases', 'antigen_specificity'),c('three'),'./result/Original/seen/', 'Epitope')
process_original(c('McPAS_1', 'McPAS_2', 'McPAS_3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','Epitope')

[1] "Saved File： ./result/Original/seen/three/VDJdb_1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_3_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_3_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/VDJdb_3_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/unseen/VDJdb_1_healthall_result.csv"
[1] "Saved File： ./result/Original/unseen/VDJdb_1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/unseen/VDJdb_1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/unsee

In [12]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TPBTE/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'Epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"


New names:
• `` -> `...1`


...

[1] "Saved File： ./result/pair300/seen/200/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/100/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/10/Antigen_specificity/5_1_1all_result.csv"


# SETE

In [84]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/SETE/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three'),'./result/Original/seen/', 'Epitope')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_antigen_specificityall_result.csv"


In [15]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/SETE/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'Epitope')
pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"


New names:
• `` -> `...1`


...

[1] "Saved File： ./result/pair300/seen/200/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/100/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/10/Antigen_specificity/5_1_1all_result.csv"


In [71]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/SETE/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
base_path <- "./result/top5/seen/healthy/"
top5_no_parameter(reap, name, number, base_path,'Epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2400all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2200all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2000all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1800all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1600all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1400all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1200all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1000all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_800all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_600all_result.csv"
[1] "Saved File: ./result/t

# TCRBERT

In [80]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three','seven'),'./result/Original/seen/', 'Epitope')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Orig

In [17]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'Epitope')
pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')



New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"


New names:
• `` -> `...1`


...

[1] "Saved File： ./result/pair300/seen/200/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/100/Antigen_specificity/5_1_1all_result.csv"


New names:
• `` -> `...1`


[1] "Saved File： ./result/pair300/seen/10/Antigen_specificity/5_1_1all_result.csv"


In [7]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
base_path <- "./result/top5/seen/healthy/"
top5_no_parameter(reap, name, number, base_path,'Epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_40all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_40all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_60all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_40all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_50al

# teim

In [92]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three', 'seven'),'./result/Original/seen/', 'Epitope')
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','Epitope')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Orig

In [24]:
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_4all_resu

In [40]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'Epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_4all_resu

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_4all_result.csv"
[1] "Saved

[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_4all_

In [68]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEIM/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
epochs <- c(30,50,70,90,100)
base_path <- "./result/top5/seen/healthy/"
top5(reap, name, number, epochs, base_path,'Epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_90all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_90all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_30all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_70

# ATM-TCR

In [83]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three', 'seven'),'./result/Original/seen/', 'Epitope')
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','Epitope')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Orig

In [41]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/seen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'),"./result/pair50/seen/validation/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4', '1_6', '1_8'),  "./result/pair50/unseen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1', '1_2', '1_4'), "./result/pair50/unseen/validation/",'Epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_4all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_4all_resu

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_4all_result.csv"
[1] "Saved

[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_2all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_4all_

In [61]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
epochs <- c(50,70,90,100)
base_path <- "./result/top5/seen/healthy/"
top5(reap, name, number, epochs, base_path,'Epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_90all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_90all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_50all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_70all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_90all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_100all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2400_5

# TCRGP

In [90]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three'),'./result/Original/seen/', 'Epitope')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_antigen_specificityall_result.csv"


In [42]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/")
process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), "./result/pair50/seen/test/",'epitope')
process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), "./result/pair50/seen/validation/",'Epitope')
pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1'), c('more300','300','200','100','10'), "./result/pair300/seen/",'epitope')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/patient/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/patient/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/patient/3_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/patient/4_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/patient/5_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/Antigen_specificity/1_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/Antigen_specificity/2_1_1all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/Antigen_specificity/3_1_1all_result.csv"
[1] "Saved File： ./result/pa

In [75]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/")
reap <- c('reap1', 'reap2', 'reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
base_path <- "./result/top5/seen/healthy/"
top5_TCRGP(reap, name, number, base_path,'epitope')


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_300020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_280020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_260020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_240020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_220020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_200020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_180020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_160020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_140020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_120020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_100020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_80020all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_60020all_result.csv"
[

# ERGO

In [96]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ERGO/")
process_original(c('1', '2', '3'),c('healthlstm_vdj_', 'Diseaseslstm_vdj_', 'antigen_specificitylstm_vdj_'),c('three', 'seven'),'./result/Original/seen/', 'Class')
process_original(c('lstm_vdj1', 'lstm_vdj2', 'lstm_vdj3'),c('health_', 'Diseases_', 'antigen_specificity_'), NULL, './result/Original/unseen/','Class')

process_original(c('1', '2', '3'),c('healthlstm_mcpas_', 'Diseaseslstm_mcpas_', 'antigen_specificitylstm_mcpas_'),c('three', 'seven'),'./result/Original/seen/', 'Class')
process_original(c('lstm_mcpas1', 'lstm_mcpas2', 'lstm_mcpas3'),c('health_', 'Diseases_', 'antigen_specificity_'), NULL, './result/Original/unseen/','Class')

process_original(c('1', '2', '3'),c('healthae_vdj_', 'Diseasesae_vdj_', 'antigen_specificityae_vdj_'),c('three', 'seven'),'./result/Original/seen/', 'Class')
process_original(c('ae_vdj1', 'ae_vdj2', 'ae_vdj3'),c('health_', 'Diseases_', 'antigen_specificity_'), NULL, './result/Original/unseen/','Class')

process_original(c('1', '2', '3'),c('healthae_mcpas_', 'Diseasesae_mcpas_', 'antigen_specificityae_mcpas_'),c('three', 'seven'),'./result/Original/seen/', 'Class')
process_original(c('ae_mcpas1', 'ae_mcpas2', 'ae_mcpas3'),c('health_', 'Diseases_', 'antigen_specificity_'), NULL, './result/Original/unseen/','Class')


[1] "Saved File： ./result/Original/seen/three/1_healthlstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_healthlstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseaseslstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseaseslstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificitylstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificitylstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthlstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_healthlstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseaseslstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseaseslstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificitylstm_vdj_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificitylstm_vdj_all_resu

[1] "Saved File： ./result/Original/unseen/ae_mcpas1_health_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas1_Diseases_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas1_antigen_specificity_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas2_health_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas2_Diseases_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas2_antigen_specificity_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas3_health_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas3_Diseases_all_result.csv"
[1] "Saved File： ./result/Original/unseen/ae_mcpas3_antigen_specificity_all_result.csv"


In [105]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ERGO/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_', '1_6ae_', '1_8ae_'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_'), "./result/pair50/seen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_', '1_6ae_', '1_8ae_'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_'),"./result/pair50/seen/validation/",'Epitope')

process_files_ERGO_ae(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_', '1_6ae_', '1_8ae_'),  "./result/pair50/unseen/test/",'Epitope')
process_files_ERGO_ae(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_'), "./result/pair50/unseen/test/",'Epitope')
process_files_ERGO_ae(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_', '1_6ae_', '1_8ae_'),  "./result/pair50/unseen/validation/",'Epitope')
process_files_ERGO_ae(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_'), "./result/pair50/unseen/validation/",'Epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_4ae_all_result.csv"
[1] "Saved File： ./result

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1

In [113]:
process_files_ERGO_ae(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_', '1_6ae_', '1_8ae_'),  "./result/pair50/unseen/test/",'Epitope')
process_files_ERGO_ae(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_'), "./result/pair50/unseen/test/",'Epitope')
process_files_ERGO_ae(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_', '1_6ae_', '1_8ae_'),  "./result/pair50/unseen/validation/",'Epitope')
process_files_ERGO_ae(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1ae_', '1_2ae_', '1_4ae_'), "./result/pair50/unseen/validation/",'Epitope')

[1] "Saved File： ./result/pair50/unseen/test/healthy/1_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/2_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/3_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/4_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/5_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/1_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/2_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/3_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/4_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/5_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/1_1_4ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/2_1_4ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/test/healthy/3_1_4ae_all_result.csv"

[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_2ae_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation

In [107]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ERGO/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_', '1_6lstm_', '1_8lstm_'), "./result/pair50/seen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_'), "./result/pair50/seen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_', '1_6lstm_', '1_8lstm_'), "./result/pair50/seen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_'),"./result/pair50/seen/validation/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_', '1_6lstm_', '1_8lstm_'),  "./result/pair50/unseen/test/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_'), "./result/pair50/unseen/test/",'Epitope')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_', '1_6lstm_', '1_8lstm_'),  "./result/pair50/unseen/validation/",'Epitope')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1lstm_', '1_2lstm_', '1_4lstm_'), "./result/pair50/unseen/validation/",'Epitope')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1lstm_'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Epitope')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_4lstm_all_result.csv"

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/A

[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_2lstm_all_result.csv"
[1] "Saved File： ./result/pair

In [8]:
#top5
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ERGO/")
reap <- c('reap1', 'reap2','reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
epochs <- c(20,30,40,50,60)
base_path <- "./result/top5/seen/healthy/"
top5ERGO(reap, name, number, epochs, base_path,'Epitope','ae_')
epochs <- c(20,40,60,80,100)
top5ERGO(reap, name, number, epochs, base_path,'Epitope','_lstm_')
top5ERGO(reap, name, number, epochs, base_path,'Epitope','lstm_')

[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_20ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_30ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_40ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_50ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_60ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_20ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_30ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_40ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_50ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_60ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_20ae_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_30ae_all_result.csv"
[1] "Saved File: ./result/to

[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_20_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_40_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_60_lstm_all_result.csv"


Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap1_top1_3000_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap1_top1_3000_100_lstm_probability.csv”


[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_20_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_40_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_60_lstm_all_result.csv"


Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap1_top1_2800_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap1_top1_2800_100_lstm_probability.csv”


...

Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_1800_60_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_1800_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_1800_100_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_1600_20_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_1600_40_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm

Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_50_60_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_50_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top3_50_100_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_3000_20_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_3000_40_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“

Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_1400_60_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_1400_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_1400_100_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_1200_20_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top4_1200_40_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm

Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_2800_60_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_2800_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_2800_100_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_2600_20_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_2600_40_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm

Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_1000_60_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_1000_80_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_1000_100_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_800_20_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"):
“The file does not exist: ./result/top5/seen/healthy/reap5_top5_800_40_lstm_probability.csv”
Warning message in top5ERGO(reap, name, number, epochs, base_path, "Epitope", "_lstm_"

In [30]:
top5ERGO <- function(reap, name, number, n_estimators, base_path,column,model) {
  for (i in reap) {
    for (j in name) {
      for (k in number) {
        for (estimators in n_estimators) {
          path <- paste0(base_path, i, '_', j, '_', k, "_", estimators,model)
          data_path <- paste0(path, "probability.csv")
          if (file.exists(data_path)) {
            data <- read.csv(data_path, stringsAsFactors = FALSE)
            path <- paste0(base_path, i, '_', j, '_', k, "_", estimators, "_",model)
            calculate_top5_auc(data, path, column) 
          } else {
            warning("The file does not exist: ", data_path)
          }
        }
      }
    }
  }
}
epochs <- c(20,40,60,80,100,120)
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ERGO/")
reap <- c('reap1', 'reap2','reap3', 'reap4', 'reap5')
name <- c('top1', 'top2', 'top3', 'top4', 'top5')
number <- c('3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50')
top5ERGO(reap, name, number, epochs, base_path,'Epitope','lstm_')

[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_3000_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2800_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2600_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2400_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2200_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_2000_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1800_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1600_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1400_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1200_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_1000_120_lstm_all_result.csv"
[1] "Saved File: ./result/top5/seen/healthy/reap1_top1_800_120_ls

# DLpTCR

In [125]:
##Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/")
process_original(c('1', '2', '3'),c('health_CNN_', 'Diseases_CNN_', 'antigen_specificity_CNN_'),c('three', 'seven'),'./result/Original/seen/', 'Epitope')
process_original(c('1', '2', '3'),c('health_CNN_', 'Diseases_CNN_', 'antigen_specificity_CNN_'), NULL, './result/Original/unseen/','Epitope')

process_original(c('1', '2', '3'),c('health_FULL_', 'Diseases_FULL_', 'antigen_specificity_FULL_'),c('three', 'seven'),'./result/Original/seen/', 'Epitope')
process_original(c('1', '2', '3'),c('health_FULL_', 'Diseases_FULL_', 'antigen_specificity_FULL_'), NULL, './result/Original/unseen/','Epitope')

process_original(c('1', '2', '3'),c('health_RESNET_', 'Diseases_RESNET_', 'antigen_specificity_RESNET_'),c('three', 'seven'),'./result/Original/seen/', 'Epitope')
process_original(c('1', '2', '3'),c('health_RESNET_', 'Diseases_RESNET_', 'antigen_specificity_RESNET_'), NULL, './result/Original/unseen/','Epitope')


[1] "Saved File： ./result/Original/seen/three/1_health_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_health_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseases_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseases_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificity_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificity_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_health_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_health_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseases_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseases_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificity_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificity_CNN_all_result.csv"
[1] "Saved File： ./result/Original/seen/

In [126]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_', '1_6CNN_pca20_', '1_8CNN_pca20_'), "./result/pair50/seen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_'), "./result/pair50/seen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_', '1_6CNN_pca20_', '1_8CNN_pca20_'), "./result/pair50/seen/validation/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_'),"./result/pair50/seen/validation/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_', '1_6CNN_pca20_', '1_8CNN_pca20_'),  "./result/pair50/unseen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_'), "./result/pair50/unseen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_', '1_6CNN_pca20_', '1_8CNN_pca20_'),  "./result/pair50/unseen/validation/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_', '1_2CNN_pca20_', '1_4CNN_pca20_'), "./result/pair50/unseen/validation/",'Class')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1CNN_pca20_'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Class')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4CNN_pca20_all_result.csv"
[1] "Saved File：

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_2CNN_pca20_all_result.csv"


[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2CNN_pca20_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_2CNN_pc

In [127]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_', '1_6RESNET_pca10_', '1_8RESNET_pca10_'), "./result/pair50/seen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_'), "./result/pair50/seen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_', '1_6RESNET_pca10_', '1_8RESNET_pca10_'), "./result/pair50/seen/validation/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_'),"./result/pair50/seen/validation/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_', '1_6RESNET_pca10_', '1_8RESNET_pca10_'),  "./result/pair50/unseen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_'), "./result/pair50/unseen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_', '1_6RESNET_pca10_', '1_8RESNET_pca10_'),  "./result/pair50/unseen/validation/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_', '1_2RESNET_pca10_', '1_4RESNET_pca10_'), "./result/pair50/unseen/validation/",'Class')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1RESNET_pca10_'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Class')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4RESNET_pc

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_

[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2RESNET_pca10_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Anti

In [128]:
#Retraining predictions
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/")
process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_', '1_6FULL_pca18_', '1_8FULL_pca18_'), "./result/pair50/seen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_'), "./result/pair50/seen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_', '1_6FULL_pca18_', '1_8FULL_pca18_'), "./result/pair50/seen/validation/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_'),"./result/pair50/seen/validation/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_', '1_6FULL_pca18_', '1_8FULL_pca18_'),  "./result/pair50/unseen/test/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_'), "./result/pair50/unseen/test/",'Class')

process_files(c('healthy', 'patient'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_', '1_6FULL_pca18_', '1_8FULL_pca18_'),  "./result/pair50/unseen/validation/",'Class')
process_files(c('Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_', '1_2FULL_pca18_', '1_4FULL_pca18_'), "./result/pair50/unseen/validation/",'Class')

pair300_process_files(c('healthy', 'patient','Antigen_specificity'), c('1', '2', '3', '4', '5'), c('1_1FULL_pca18_'), c('more300','300','200','100','10'), "./result/pair300/seen/",'Class')

[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/3_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/4_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/5_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/1_1_4FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/test/healthy/2_1_4FULL_pca18_all_result.csv"
[1] 

[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/1_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/2_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/3_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/4_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/seen/validation/Antigen_specificity/5_1_2FULL_pca18_all_re

[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_1_1FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/1_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/2_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/3_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/4_1_2FULL_pca18_all_result.csv"
[1] "Saved File： ./result/pair50/unseen/validation/Antigen_specificity/5_

# ImRex

In [130]:
#Original prediction
setwd("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ImRex/")
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'),c('three','seven'),'./result/Original/seen/', 'Class')
process_original(c('1', '2', '3'),c('health', 'Diseases', 'antigen_specificity'), NULL, './result/Original/unseen/','Class')

[1] "Saved File： ./result/Original/seen/three/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/1_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_healthall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_Diseasesall_result.csv"
[1] "Saved File： ./result/Original/seen/three/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/seven/2_antigen_specificityall_result.csv"
[1] "Saved File： ./result/Original/seen/three/3_healthall_result.csv"
[1] "Saved File： ./result/Orig